In [7]:
import warnings
warnings.filterwarnings('ignore','.*conversion.*')

import os
import h5py
from config import opt
import main
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import (interact, interact_manual, interactive, fixed)
from evaluation import load_result
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import *

if opt.hdf5:
    from datasets import Train_Dataset_HDF5 as Train_Dataset
    from datasets import Test_Dataset_HDF5 as Test_Dataset
else:
    from datasets import Train_Dataset_IMAGE as Train_D
    from datasets import Test_Dataset_IMAGE as Test_Dataset
    
dataset_name = widgets.Dropdown(
                        options=['Market1501', 'DukeMTMC'],
                        value='Market1501',
                        description='Dataset:',
                        disabled=False,
                    )
model = widgets.Dropdown(
                    options=['ResNet50', 'DenseNet121'],
                    value='ResNet50',
                    description='Model:',
                    disabled=False,
                )

re_ranking = widgets.Checkbox(
                value=True,
                description='Re_Ranking:',
                disabled=False
            )

load_features = widgets.Checkbox(
                value=True,
                description='Load Extracted Features:',
                disabled=False
            )


train_button = widgets.Button(description="Train")
test_button = widgets.Button(description='Test')
items = [train_button,test_button]
button = widgets.Box(items)

out = widgets.Output()
def click_train(b):
    with out:
        clear_output()
        opt.dataset_name = dataset_name.value
        opt.model = model.value
        main.train()
def click_test(b):
    with out:
        clear_output()
        opt.dataset_name = dataset_name.value
        opt.model = model.value
        opt.re_ranking = re_ranking.value
        opt.load_features = load_features.value
        main.test()
train_button.on_click(click_train)
test_button.on_click(click_test)

widgets.VBox([dataset_name,model,re_ranking, load_features, button, out])

VBox(children=(Dropdown(description='Dataset:', options=('Market1501', 'DukeMTMC'), value='Market1501'), Dropdown(description='Model:', options=('ResNet50', 'DenseNet121'), value='ResNet50'), Checkbox(value=True, description='Re_Ranking:'), Checkbox(value=True, description='Load Extracted Features:'), Box(children=(Button(description='Train', style=ButtonStyle()), Button(description='Test', style=ButtonStyle()))), Output()))

In [79]:
result,CMC,mAP = load_result()
query_name = result['query_name']
query_ids = []
for name in query_name:
    id = name.split('_')[0]
    if not id in query_ids:
        query_ids.append(id)

person_id = widgets.Dropdown(options= query_ids,
                             value=query_ids[0],
                             description='Person ID')

person_name = widgets.Dropdown(options= [name for name in query_name if person_id.value in name[:4]],
                               value=[name for name in query_name if person_id.value in name[:4]][0],
                               description='Person Name')

index = widgets.IntSlider(value=0,
                          min=0,
                          max=len(result['query_name']),
                          description='Index'
                         )

R=widgets.IntSlider(value=5,min=0,max=20)
size = widgets.IntSlider(value=20,min=10,max=40)
re_ranking = widgets.Checkbox(value=True,description='Show Re_Ranking:')

def update_name(*args):
    person_name.options = [name for name in query_name if person_id.value in name[:4]]
def update_index_from_id(*args):
    index.value = query_name.index([name for name in query_name if person_id.value in name[:4]][0])
def update_index_from_name(*args):
    index.value = query_name.index(person_name.value)
def updata_id_from_index(*args):
    person_id.value = query_name[index.value][:4]
def updata_name_from_index(*args):
    person_name.value = query_name[index.value]

person_id.observe(update_name, 'value')
person_id.observe(update_index_from_id, 'value')
person_name.observe(update_index_from_name, 'value')
index.observe(updata_id_from_index, 'value')
index.observe(updata_name_from_index, 'value')

def result_demo(person_id, person_name,index,R,size, re_ranking):
    if not re_ranking:
        result,CMC,mAP = load_result()
        ranking = result['ranking']
        query_name = result['query_name']
        gallery_name = result['gallery_name']
        if opt.hdf5:
            hdf5_path = os.path.join(opt.data_dir,opt.dataset_name,opt.dataset_name+'.hdf5')
            f = h5py.File(hdf5_path,'r')
        dataset_dir = os.path.join(opt.data_dir,opt.dataset_name)
        query_label = query_name[index].split('_')[0]
        fig, img = plt.subplots(1,R+1,figsize=(size,size))
        for i in range(0,R+1):
            if i == 0:
                if opt.hdf5:
                    img[i].imshow(f['query'][query_name[index]]['img'])
                else:
                    img[i].imshow(mpimg.imread(os.path.join(dataset_dir,'query', query_name[index]+'.jpg')))
                img[i].set_title('Query Image \n ID:'+query_label)
            else:
                if opt.hdf5:
                    img[i].imshow(f['gallery'][gallery_name[int(ranking[index][i-1])]]['img'])
                else:
                    img[i].imshow(mpimg.imread(os.path.join(dataset_dir,'bounding_box_test', gallery_name[int(ranking[index][i-1])]+'.jpg')))
                gallery_label = gallery_name[int(ranking[index][i-1])].split('_')[0]
                if i == 1:
                    img[i].set_title('Gallery Images \n ID:'+gallery_label)
                else:
                    img[i].set_title('ID:'+gallery_label)

                if(gallery_label== query_label):
                    autoAxis = img[i].axis()
                    rec = Rectangle((autoAxis[0]-0.7,autoAxis[2]-0.2),(autoAxis[1]-autoAxis[0])+1,(autoAxis[3]-autoAxis[2])+0.4,fill=False,lw=2,color='red')
                    rec = img[i].add_patch(rec)
                    rec.set_clip_on(False)
            img[i].axis('off')
        plt.show()
    else:
        opt.re_ranking = False
        original_result,_,_ = load_result()
        original_ranking = original_result['ranking']
        opt.re_ranking = True
        rerank_result,_,_ = load_result()
        rerank_ranking = rerank_result['ranking']
        
        query_name = original_result['query_name']
        gallery_name = original_result['gallery_name']
        
        if opt.hdf5:
            hdf5_path = os.path.join(opt.data_dir,opt.dataset_name,opt.dataset_name+'.hdf5')
            f = h5py.File(hdf5_path,'r')
        dataset_dir = os.path.join(opt.data_dir,opt.dataset_name)
        query_label = query_name[index].split('_')[0]
        fig, img = plt.subplots(2,R+1,figsize=(size,size/1.5))
        for i in range(0,R+1):
            if i == 0:
                if opt.hdf5:
                    img[0,i].imshow(f['query'][query_name[index]]['img'])
                    h,w,c = f['query'][query_name[0]]['img'].shape
                    img[1,i].imshow(np.zeros((h,w))+255,cmap=matplotlib.cm.gray, vmin=0, vmax=255)
                else:
                    img[0,i].imshow(mpimg.imread(os.path.join(dataset_dir,'query', query_name[index]+'.jpg')))
                    img[1,i].imshow(mpimg.imread(os.path.join(dataset_dir,'query', query_name[index]+'.jpg')))
                img[0,i].set_title('Query Image \n ID:'+query_label)
            else:
                if opt.hdf5:
                    img[0,i].imshow(f['gallery'][gallery_name[int(original_ranking[index][i-1])]]['img'])
                    img[1,i].imshow(f['gallery'][gallery_name[int(rerank_ranking[index][i-1])]]['img'])
                else:
                    img[0,i].imshow(mpimg.imread(os.path.join(dataset_dir,'bounding_box_test', gallery_name[int(original_ranking[index][i-1])]+'.jpg')))
                    img[1,i].imshow(mpimg.imread(os.path.join(dataset_dir,'bounding_box_test', gallery_name[int(rerank_ranking[index][i-1])]+'.jpg')))
                    
                gallery_label = gallery_name[int(original_ranking[index][i-1])].split('_')[0]
                rerank_gallery_label = gallery_name[int(rerank_ranking[index][i-1])].split('_')[0]
                if i == 1:
                    img[0,i].set_title('Original Images \n ID:'+gallery_label)
                    img[1,i].set_title('Re-Ranked Images \n ID:'+rerank_gallery_label)
                else:
                    img[0,i].set_title('ID:'+gallery_label)
                    img[1,i].set_title('ID:'+rerank_gallery_label)
                    

                if(gallery_label== query_label):
                    autoAxis = img[0,i].axis()
                    rec = Rectangle((autoAxis[0]-0.7,autoAxis[2]-0.2),(autoAxis[1]-autoAxis[0])+1,(autoAxis[3]-autoAxis[2])+0.4,fill=False,lw=2,color='red')
                    rec = img[0,i].add_patch(rec)
                    rec.set_clip_on(False)
                
                if(rerank_gallery_label== query_label):
                    autoAxis = img[1,i].axis()
                    rec = Rectangle((autoAxis[0]-0.7,autoAxis[2]-0.2),(autoAxis[1]-autoAxis[0])+1,(autoAxis[3]-autoAxis[2])+0.4,fill=False,lw=2,color='red')
                    rec = img[1,i].add_patch(rec)
                    rec.set_clip_on(False)
            img[0,i].axis('off')
            img[1,i].axis('off')
        plt.subplots_adjust(wspace=0.1, hspace=0.1)
        plt.show()
        

demo = interact_manual(result_demo,
         person_id = person_id,
         person_name = person_name,
         index = index,
         R = R, 
         size = size,
                       
         re_ranking = re_ranking)

interactive(children=(Dropdown(description='Person ID', options=('0001', '0003', '0004', '0005', '0006', '0008', '0009', '0013', '0014', '0015', '0016', '0017', '0018', '0019', '0021', '0024', '0025', '0026', '0029', '0031', '0033', '0034', '0036', '0038', '0039', '0040', '0041', '0044', '0045', '0049', '0050', '0051', '0054', '0055', '0058', '0060', '0061', '0062', '0063', '0066', '0071', '0072', '0073', '0074', '0075', '0078', '0080', '0083', '0085', '0087', '0089', '0091', '0092', '0094', '0096', '0101', '0102', '0103', '0109', '0112', '0113', '0119', '0120', '0124', '0126', '0128', '0130', '0131', '0133', '0137', '0138', '0144', '0145', '0146', '0147', '0152', '0153', '0154', '0155', '0156', '0157', '0161', '0163', '0165', '0168', '0170', '0171', '0174', '0182', '0183', '0186', '0187', '0188', '0189', '0191', '0192', '0194', '0196', '0198', '0200', '0203', '0205', '0207', '0210', '0213', '0215', '0217', '0218', '0219', '0220', '0226', '0227', '0228', '0229', '0230', '0231', '0233', '0235', '0238', '0240', '0244', '0246', '0247', '0252', '0253', '0256', '0257', '0258', '0260', '0262', '0263', '0265', '0267', '0270', '0271', '0274', '0275', '0278', '0280', '0283', '0284', '0285', '0286', '0288', '0289', '0290', '0291', '0292', '0293', '0294', '0295', '0300', '0302', '0304', '0305', '0310', '0311', '0312', '0315', '0316', '0319', '0320', '0322', '0329', '0330', '0334', '0336', '0337', '0342', '0343', '0345', '0346', '0351', '0353', '0355', '0356', '0360', '0362', '0363', '0364', '0365', '0366', '0372', '0373', '0378', '0381', '0387', '0388', '0391', '0395', '0396', '0400', '0401', '0405', '0406', '0412', '0416', '0417', '0418', '0422', '0425', '0426', '0428', '0431', '0436', '0438', '0439', '0440', '0443', '0447', '0448', '0452', '0453', '0454', '0455', '0458', '0460', '0461', '0462', '0463', '0465', '0467', '0469', '0471', '0473', '0474', '0476', '0478', '0479', '0483', '0487', '0488', '0489', '0490', '0492', '0493', '0495', '0497', '0498', '0501', '0502', '0504', '0505', '0506', '0507', '0510', '0511', '0512', '0514', '0520', '0521', '0523', '0526', '0527', '0530', '0531', '0532', '0533', '0535', '0538', '0541', '0542', '0543', '0544', '0548', '0550', '0553', '0559', '0560', '0561', '0562', '0567', '0568', '0569', '0574', '0576', '0577', '0578', '0580', '0582', '0583', '0585', '0587', '0590', '0591', '0595', '0598', '0600', '0601', '0602', '0607', '0608', '0609', '0610', '0617', '0618', '0621', '0624', '0625', '0626', '0627', '0631', '0632', '0634', '0638', '0643', '0644', '0646', '0650', '0651', '0654', '0664', '0668', '0671', '0672', '0675', '0678', '0679', '0680', '0684', '0686', '0687', '0690', '0691', '0692', '0693', '0694', '0695', '0698', '0699', '0710', '0713', '0715', '0716', '0717', '0719', '0720', '0721', '0722', '0723', '0725', '0727', '0728', '0731', '0732', '0735', '0736', '0737', '0740', '0743', '0745', '0746', '0747', '0750', '0751', '0753', '0756', '0758', '0763', '0764', '0768', '0769', '0770', '0771', '0775', '0776', '0777', '0778', '0783', '0784', '0786', '0789', '0790', '0791', '0794', '0797', '0798', '0799', '0800', '0801', '0804', '0805', '0807', '0808', '0811', '0812', '0813', '0815', '0817', '0819', '0822', '0824', '0825', '0827', '0829', '0831', '0834', '0835', '0836', '0838', '0841', '0845', '0846', '0847', '0852', '0853', '0856', '0858', '0860', '0861', '0865', '0866', '0867', '0869', '0870', '0873', '0874', '0877', '0878', '0880', '0881', '0884', '0888', '0889', '0897', '0899', '0906', '0908', '0909', '0910', '0911', '0912', '0913', '0916', '0918', '0920', '0921', '0922', '0923', '0924', '0925', '0927', '0928', '0929', '0931', '0932', '0934', '0935', '0937', '0938', '0944', '0949', '0950', '0951', '0956', '0959', '0960', '0964', '0965', '0966', '0968', '0974', '0977', '0978', '0980', '0981', '0983', '0985', '0989', '0993', '0996', '1005', '1006', '1008', '1009', '1013', '1014', '1015', '1016', '1020', '1021', '1022', '1024', '1026', '1028', '1029', '1034', '1035', '1036', '1037', '1040', '1042',